In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json
/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json
/kaggle/input/arc-prize-2024/arc-agi_evaluation_challenges.json
/kaggle/input/arc-prize-2024/sample_submission.json
/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json
/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json


In [2]:
import json

training_solutions_path = '/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json'
evaluation_solutions_path = '/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json'
evaluation_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_evaluation_challenges.json'
sample_submission_path = '/kaggle/input/arc-prize-2024/sample_submission.json'
training_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json'
test_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json'

#function to load JSON data
def load_json_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

#load each dataset
training_solutions = load_json_data(training_solutions_path)
evaluation_solutions = load_json_data(evaluation_solutions_path)
evaluation_challenges = load_json_data(evaluation_challenges_path)
sample_submission = load_json_data(sample_submission_path)
training_challenges = load_json_data(training_challenges_path)
test_challenges = load_json_data(test_challenges_path)

In [3]:
#inspect the structure of the data
def inspect_data(data, name):
    print(f"Inspecting {name}:")
    if isinstance(data, list):
        print(f"Number of items: {len(data)}")
        if len(data) > 0:
            print(f"Example item: {data[0]}")
    elif isinstance(data, dict):
        print(f"Number of keys: {len(data.keys())}")
        if len(data.keys()) > 0:
            first_key = list(data.keys())[0]
            print(f"Example item under key '{first_key}': {data[first_key]}")
    else:
        print("Unknown data type")
    print("\n")

inspect_data(training_solutions, "Training Solutions")
inspect_data(evaluation_solutions, "Evaluation Solutions")
inspect_data(evaluation_challenges, "Evaluation Challenges")
inspect_data(sample_submission, "Sample Submission")
inspect_data(training_challenges, "Training Challenges")
inspect_data(test_challenges, "Test Challenges")

Inspecting Training Solutions:
Number of keys: 400
Example item under key '007bbfb7': [[[7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 7, 0, 0, 0, 0, 7, 7, 0], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 7, 0, 0, 0, 0, 7, 7, 0], [7, 0, 7, 7, 0, 7, 0, 0, 0], [7, 0, 7, 7, 0, 7, 0, 0, 0], [7, 7, 0, 7, 7, 0, 0, 0, 0]]]


Inspecting Evaluation Solutions:
Number of keys: 400
Example item under key '00576224': [[[3, 2, 3, 2, 3, 2], [7, 8, 7, 8, 7, 8], [2, 3, 2, 3, 2, 3], [8, 7, 8, 7, 8, 7], [3, 2, 3, 2, 3, 2], [7, 8, 7, 8, 7, 8]]]


Inspecting Evaluation Challenges:
Number of keys: 400
Example item under key '00576224': {'test': [{'input': [[3, 2], [7, 8]]}], 'train': [{'input': [[8, 6], [6, 4]], 'output': [[8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4], [6, 8, 6, 8, 6, 8], [4, 6, 4, 6, 4, 6], [8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4]]}, {'input': [[7, 9], [4, 3]], 'output': [[7, 9, 7, 9, 7, 9], [4, 3, 4, 3, 4, 3], [9, 7, 9, 7, 9, 7], [3, 4, 3, 4, 3, 4], [7, 9, 7, 9, 7, 

In [4]:
def get_max_grid_size(challenges, solutions):
    max_input_height, max_input_width = 0, 0
    max_output_height, max_output_width = 0, 0
    
    for key in challenges.keys():
        challenge = challenges[key]
        for example in challenge['train']:
            input_grid = example['input']
            output_grid = example['output']
            max_input_height = max(max_input_height, len(input_grid))
            max_input_width = max(max_input_width, len(input_grid[0]))
            max_output_height = max(max_output_height, len(output_grid))
            max_output_width = max(max_output_width, len(output_grid[0]))
        for test_case in challenge['test']:
            test_input = test_case['input']
            max_input_height = max(max_input_height, len(test_input))
            max_input_width = max(max_input_width, len(test_input[0]))
            #assuming test_output size can be derived similarly

    return max_input_height, max_input_width, max_output_height, max_output_width

max_input_height, max_input_width, max_output_height, max_output_width = get_max_grid_size(training_challenges, training_solutions)
print(f"Max input size: {max_input_height}x{max_input_width}, Max output size: {max_output_height}x{max_output_width}")

Max input size: 30x30, Max output size: 30x30


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class ARCDataset(Dataset):
    def __init__(self, challenges, solutions, max_size, transform=None):
        self.data = []
        self.max_size = max_size
        self.transform = transform
        for key in challenges.keys():
            challenge = challenges[key]
            solution = solutions[key]
            for example in challenge['train']:
                input_grid = example['input']
                output_grid = example['output']
                self.data.append((input_grid, output_grid))
            for test_case in challenge['test']:
                test_input = test_case['input']
                #use the corresponding solution as the target output
                test_output = solution[len(self.data) % len(solution)]
                self.data.append((test_input, test_output))

    def __len__(self):
        return len(self.data)

    def pad_grid(self, grid):
        padded_grid = np.zeros(self.max_size)
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                padded_grid[i][j] = grid[i][j]
        return padded_grid

    def __getitem__(self, idx):
        input_grid, output_grid = self.data[idx]
        input_grid = self.pad_grid(input_grid)
        output_grid = self.pad_grid(output_grid)
        input_grid = torch.tensor(input_grid, dtype=torch.float32).unsqueeze(0)
        output_grid = torch.tensor(output_grid, dtype=torch.float32).unsqueeze(0)
        if self.transform:
            input_grid = self.transform(input_grid)
            output_grid = self.transform(output_grid)
        return input_grid, output_grid

#define the maximum size
max_size = (30, 30)

#create datasets with padding
transform = transforms.Compose([transforms.Lambda(lambda x: x)])  #no additional transform needed
training_dataset = ARCDataset(training_challenges, training_solutions, max_size, transform=transform)
evaluation_dataset = ARCDataset(evaluation_challenges, evaluation_solutions, max_size, transform=transform)

#create dataloaders
training_loader = DataLoader(training_dataset, batch_size=1, shuffle=True)
evaluation_loader = DataLoader(evaluation_dataset, batch_size=1, shuffle=False)

In [6]:
def preprocess_input(x, normalize=False, mean=0.5, std=0.5):
    """
    Preprocesses the input tensor `x` and returns it along with dimensions needed by subsequent modules.
    
    Args:
    - x (torch.Tensor): Input tensor with shape (batch_size, channels, height, width).
    - normalize (bool): Whether to normalize the input tensor.
    - mean (float or list): Mean for normalization (scalar or list for each channel).
    - std (float or list): Standard deviation for normalization (scalar or list for each channel).
    
    Returns:
    - x (torch.Tensor): Preprocessed input tensor.
    - input_channels (int): Number of channels in the input tensor.
    - input_height (int): Height of the input tensor.
    - input_width (int): Width of the input tensor.
    
    Raises:
    - ValueError: If the input tensor `x` does not have the expected number of dimensions or channels.
    """
    #ensure the input tensor has 4 dimensions (batch_size, channels, height, width)
    if x.dim() != 4:
        raise ValueError(f"Expected input tensor with 4 dimensions (batch_size, channels, height, width), got {x.dim()} dimensions instead.")
    
    #check the number of channels
    input_channels = x.size(1)
    
    #ensure that the number of channels is 1 or 3 (grayscale or RGB)
    if input_channels not in [1, 3]:
        raise ValueError(f"Expected input tensor with 1 or 3 channels, got {input_channels} channels instead.")
    
    #extract input dimensions
    input_height = x.size(2)
    input_width = x.size(3)
    
    #normalize the input tensor if required
    if normalize:
        if isinstance(mean, list) and isinstance(std, list):
            if len(mean) != input_channels or len(std) != input_channels:
                raise ValueError("Mean and std lists must have the same length as the number of channels.")
            mean = torch.tensor(mean).view(1, input_channels, 1, 1)
            std = torch.tensor(std).view(1, input_channels, 1, 1)
        else:
            mean = torch.tensor([mean] * input_channels).view(1, input_channels, 1, 1)
            std = torch.tensor([std] * input_channels).view(1, input_channels, 1, 1)
        x = (x - mean) / std
    
    return x, input_channels, input_height, input_width

#example usage
x1 = torch.randn(8, 1, 28, 28)  # Grayscale input tensor
x2 = torch.randn(8, 3, 32, 32)  # RGB input tensor

#without normalization
preprocessed_x1, channels1, height1, width1 = preprocess_input(x1)
print(f"Preprocessed tensor shape: {preprocessed_x1.shape}")
print(f"Channels: {channels1}, Height: {height1}, Width: {width1}")

#with normalization
preprocessed_x2, channels2, height2, width2 = preprocess_input(x2, normalize=True, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
print(f"Preprocessed tensor shape: {preprocessed_x2.shape}")
print(f"Channels: {channels2}, Height: {height2}, Width: {width2}")

Preprocessed tensor shape: torch.Size([8, 1, 28, 28])
Channels: 1, Height: 28, Width: 28
Preprocessed tensor shape: torch.Size([8, 3, 32, 32])
Channels: 3, Height: 32, Width: 32


In [7]:
import torch.nn as nn
import torch.nn.functional as F

#perception module
class PerceptionModule(nn.Module):
    def __init__(self, in_channels):
        super(PerceptionModule, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        
        #use Adaptive Average Pooling to handle varying input sizes
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))  #output size (1, 1)
        
        #fully connected layer
        self.fc = nn.Linear(64, 512)  #adjusted input size to 64 from adaptive pooling

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)  #flatten the tensor
        x = self.fc(x)
        return x

In [8]:
#test perception module
batch_size = 8
input_channels = 3
input_height = 32
input_width = 32

model = PerceptionModule(in_channels=input_channels)
dummy_input = torch.randn(batch_size, input_channels, input_height, input_width)
output = model(dummy_input)

print(f"Perception module output shape : {output.shape}")  #should print torch.Size([8, 512])

Perception module output shape : torch.Size([8, 512])


In [9]:
#memory module
class MemoryModule(nn.Module):
    def __init__(self, input_size, output_size=512, normalize=False, mean=0.5, std=0.5):
        super(MemoryModule, self).__init__()
        self.normalize = normalize
        self.mean = mean
        self.std = std
        
        #define a fully connected layer with configurable input and output sizes
        self.fc = nn.Linear(input_size, output_size)
        
    def preprocess_input(self, x):
        if self.normalize:
            #normalize input if required
            x = (x - self.mean) / self.std
        return x

    def forward(self, x):
        #apply preprocessing
        x = self.preprocess_input(x)
        
        #ensure the input is of the correct shape
        x = x.view(x.size(0), -1)
        
        #forward pass through the fully connected layer
        x = self.fc(x)
        return x

In [10]:
#test memory module
input_size = 784 
output_size = 512
batch_size = 8

dummy_input = torch.randn(batch_size, input_size)

memory_module = MemoryModule(input_size=input_size, output_size=output_size, normalize=True)
output = memory_module(dummy_input)

print(f"Memory Module output shape: {output.shape}") #should print torch.Size([8, 512])

Memory Module output shape: torch.Size([8, 512])


In [11]:
#creativity module
class CreativityModule(nn.Module):
    def __init__(self, input_size, latent_size=128):
        super(CreativityModule, self).__init__()
        self.fc1 = nn.Linear(input_size, latent_size)
        self.fc2 = nn.Linear(latent_size, input_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [12]:
#test creativity module
input_size = 784 
latent_size = 128
batch_size = 8

dummy_input = torch.randn(batch_size, input_size)

creativity_module = CreativityModule(input_size=input_size, latent_size=latent_size)
output = creativity_module(dummy_input)

print(f"Creativity Module output shape: {output.shape}")  #should print torch.Size([8, 784])

Creativity Module output shape: torch.Size([8, 784])


In [13]:
#reasoning module
class ReasoningModule(nn.Module):
    def __init__(self, input_size, output_size):
        super(ReasoningModule, self).__init__()
        self.fc1 = nn.Linear(input_size, 128) 
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, output_size)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [14]:
#test reasoning module
input_size = 512
output_size = 10 
batch_size = 8

reasoning_module = ReasoningModule(input_size=input_size, output_size=output_size)

dummy_input = torch.randn(batch_size, input_size)
output = reasoning_module(dummy_input)
print(f"Reasoning Module output shape: {output.shape}")

Reasoning Module output shape: torch.Size([8, 10])


In [15]:
#compassionate module
class CompassionateModule(nn.Module):
    def __init__(self, input_size):
        super(CompassionateModule, self).__init__()
        self.fc1 = nn.Linear(input_size, 128) 
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)  
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [16]:
#test compassionate module
input_size = 512
batch_size = 8

compassionate_module = CompassionateModule(input_size=input_size)

dummy_input = torch.randn(batch_size, input_size)

output = compassionate_module(dummy_input)
print(f"Compassionate Module output shape: {output.shape}")

Compassionate Module output shape: torch.Size([8, 128])


In [17]:
#bias detection and mitigation module
class BiasDetectionMitigationModule(nn.Module):
    def __init__(self, input_size):
        super(BiasDetectionMitigationModule, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 128) 
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [18]:
#test bias detection and mitigation module
input_size = 512 
module = BiasDetectionMitigationModule(input_size)

batch_size = 8
dummy_input = torch.randn(batch_size, input_size)

output = module(dummy_input)

print(f"Dummy input shape: {dummy_input.shape}")
print(f"Output shape: {output.shape}")

Dummy input shape: torch.Size([8, 512])
Output shape: torch.Size([8, 128])


In [19]:
#planning module
class PlanningModule(nn.Module):
    def __init__(self, input_size):
        super(PlanningModule, self).__init__()
        self.fc = nn.Linear(input_size, 128) 
        
    def forward(self, x):
        x = self.fc(x)
        x = F.relu(x)  #apply ReLU activation function after linear transformation
        return x

In [20]:
#test planning module
input_size = 512
module = PlanningModule(input_size)

batch_size = 8
dummy_input = torch.randn(batch_size, input_size)

output = module(dummy_input)

print(f"Dummy input shape: {dummy_input.shape}")
print(f"Output shape: {output.shape}")

Dummy input shape: torch.Size([8, 512])
Output shape: torch.Size([8, 128])


In [21]:
#attention module
class AttentionModule(nn.Module):
    def __init__(self, input_size, output_size=128): 
        super(AttentionModule, self).__init__()
        self.fc_query = nn.Linear(input_size, output_size)
        self.fc_key = nn.Linear(input_size, output_size)
        self.fc_value = nn.Linear(input_size, output_size)
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self, x):
        batch_size = x.size(0)
        x = x.unsqueeze(1)  
        query = self.fc_query(x)
        key = self.fc_key(x)
        value = self.fc_value(x)

        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / (query.size(-1) ** 0.5)
        attention_weights = self.softmax(attention_scores)
        context_layer = torch.matmul(attention_weights, value)

        return context_layer.squeeze(1)

In [22]:
#test attention module
input_size = 512
output_size = 128
module = AttentionModule(input_size, output_size)

batch_size = 8
dummy_input = torch.randn(batch_size, input_size)

output = module(dummy_input)

print(f"Dummy input shape: {dummy_input.shape}")
print(f"Output shape: {output.shape}")

Dummy input shape: torch.Size([8, 512])
Output shape: torch.Size([8, 128])


In [23]:
#learning module
class LearningModule(nn.Module):
    def __init__(self, input_size, hidden_size1=512, hidden_size2=256):
        super(LearningModule, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, input_size) 
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [24]:
#test learning module
input_size = 784
hidden_size1 = 512
hidden_size2 = 256
module = LearningModule(input_size, hidden_size1, hidden_size2)

batch_size = 8
dummy_input = torch.randn(batch_size, input_size)

output = module(dummy_input)

print(f"Dummy input shape: {dummy_input.shape}")
print(f"Output shape: {output.shape}")

Dummy input shape: torch.Size([8, 784])
Output shape: torch.Size([8, 784])


In [25]:
#meta-reasoning module
class MetaReasoningModule(nn.Module):
    def __init__(self, input_size, output_size):
        super(MetaReasoningModule, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, output_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [26]:
#test metareasoning module
input_size = 128
output_size = 10
module = MetaReasoningModule(input_size, output_size)

batch_size = 8
dummy_input = torch.randn(batch_size, input_size)

output = module(dummy_input)

print(f"Dummy input shape: {dummy_input.shape}")
print(f"Output shape: {output.shape}")

Dummy input shape: torch.Size([8, 128])
Output shape: torch.Size([8, 10])


In [27]:
#communication module
class CommunicationModule(nn.Module):
    def __init__(self, input_size, output_size):
        super(CommunicationModule, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [28]:
#test communication module
input_size = 128 
output_size = 10 
module = CommunicationModule(input_size, output_size)

batch_size = 8
dummy_input = torch.randn(batch_size, input_size)

output = module(dummy_input)

print(f"Dummy input shape: {dummy_input.shape}")
print(f"Output shape: {output.shape}")

Dummy input shape: torch.Size([8, 128])
Output shape: torch.Size([8, 10])


In [29]:
#integration module
class IntegrationModule(nn.Module):
    def __init__(self, input_size, output_size=900):
        super(IntegrationModule, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, output_size)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [30]:
#define the Yambi model
class Yambi(nn.Module):
    """
    Yambi is an AI model designed to solve abstract reasoning tasks by integrating various cognitive abilities. 
    It consists of the following modules:
    
    1. Perception Module: Processes and understands input grids using convolutional neural networks.
    2. Memory Module: Stores and recalls information about previously seen tasks using LSTM networks.
    3. Creativity Module: Generates creative solutions and explores novel approaches using a latent space.
    4. Reasoning Module: Applies logical reasoning to infer solutions based on perceived information.
    5. Compassionate Module: Analyzes emotional context and adjusts responses accordingly.
    6. Bias Detection and Mitigation Module: Detects and mitigates biases in the reasoning output.
    7. Planning Module: Plans and sequences actions based on inferred solutions.
    8. Attention Module: Focuses on relevant information and enhances processing.
    9. Learning Module: Adapts to new data and refines existing knowledge.
    10. Meta-Reasoning Module: Reflects on the reasoning process and adjusts strategies.
    11. Communication Module: Facilitates interaction and exchange of information.
    12. Integration Module: Integrates outputs from various modules for coherent decision-making.

    """

    def __init__(self, perception_model, memory_model, creativity_model, reasoning_model,
                 compassionate_model, bias_detection_mitigation_model, planning_model,
                 attention_model, learning_model, meta_reasoning_model, communication_model,
                 integration_model):
        super(Yambi, self).__init__()

        self.perception_model = perception_model
        self.memory_model = memory_model
        self.creativity_model = creativity_model
        self.reasoning_model = reasoning_model
        self.compassionate_model = compassionate_model
        self.bias_detection_mitigation_model = bias_detection_mitigation_model
        self.planning_model = planning_model
        self.attention_model = attention_model
        self.learning_model = learning_model
        self.meta_reasoning_model = meta_reasoning_model
        self.communication_model = communication_model
        self.integration_model = integration_model
    
    def forward(self, x):
        # Perception module
        perception_output = self.perception_model(x)
        
        # Memory module
        memory_output = self.memory_model(perception_output)
        
        # Creativity module
        creativity_output = self.creativity_model(memory_output)
        
        # Reasoning module
        reasoning_output = self.reasoning_model(creativity_output)
        
        # Compassionate module
        compassionate_output = self.compassionate_model(reasoning_output)
        
        # Bias Detection and Mitigation module
        bias_detection_output = self.bias_detection_mitigation_model(memory_output)
        
        # Planning module
        planning_output = self.planning_model(reasoning_output)
        
        # Attention module
        attention_output = self.attention_model(reasoning_output)
        
        # Learning module
        learning_output = self.learning_model(reasoning_output)
        
        # Meta-Reasoning module
        meta_reasoning_output = self.meta_reasoning_model(reasoning_output)
        
        # Communication module
        communication_output = self.communication_model(reasoning_output)
        
        # Print shapes for debugging
        print("Reasoning output size:", reasoning_output.size())
        print("Compassionate output size:", compassionate_output.size())
        print("Bias Detection output size:", bias_detection_output.size())
        print("Planning output size:", planning_output.size())
        print("Attention output size:", attention_output.size())
        print("Learning output size:", learning_output.size())
        print("Meta-Reasoning output size:", meta_reasoning_output.size())
        print("Communication output size:", communication_output.size())
        
        # Integration module input
        integration_input = torch.cat([
            reasoning_output,
            compassionate_output,
            bias_detection_output,
            planning_output,
            attention_output,
            learning_output,
            meta_reasoning_output,
            communication_output
        ], dim=1)
        
        #print integration input shape for debugging
        print("Integration input size:", integration_input.size())
        
        # Integration module
        integration_output = self.integration_model(integration_input)
        
        return reasoning_output, compassionate_output, integration_output

#example usage
perception_model = PerceptionModule(in_channels=1)
memory_module = MemoryModule(input_size=512, normalize=True, mean=0.5, std=0.5)
creativity_model = CreativityModule(input_size=512)  
reasoning_model = ReasoningModule(input_size=512, output_size=128)
compassionate_model = CompassionateModule(input_size=128)  
bias_detection_mitigation_model = BiasDetectionMitigationModule(input_size=512)
planning_model = PlanningModule(input_size=128)
attention_model = AttentionModule(input_size=128, output_size=128)  
learning_model = LearningModule(input_size=128)
meta_reasoning_model = MetaReasoningModule(input_size=128, output_size=10)
communication_model = CommunicationModule(input_size=128, output_size=16)

integration_model = IntegrationModule(input_size=794, output_size=900)

#instantiate Yambi model
yambi_model = Yambi(
    perception_model, memory_module, creativity_model, reasoning_model,
    compassionate_model, bias_detection_mitigation_model, planning_model,
    attention_model, learning_model, meta_reasoning_model, communication_model,
    integration_model
)

#example forward pass
dummy_input = torch.randn(8, 1, 28, 28)
output = yambi_model(dummy_input)

#print output shapes for verification
print("Reasoning output shape:", output[0].shape)
print("Compassionate output shape:", output[1].shape)
print("Integration output shape:", output[2].shape)

Reasoning output size: torch.Size([8, 128])
Compassionate output size: torch.Size([8, 128])
Bias Detection output size: torch.Size([8, 128])
Planning output size: torch.Size([8, 128])
Attention output size: torch.Size([8, 128])
Learning output size: torch.Size([8, 128])
Meta-Reasoning output size: torch.Size([8, 10])
Communication output size: torch.Size([8, 16])
Integration input size: torch.Size([8, 794])
Reasoning output shape: torch.Size([8, 128])
Compassionate output shape: torch.Size([8, 128])
Integration output shape: torch.Size([8, 900])


In [31]:
import torch.optim as optim

#function to process labels to match model output
def process_labels(labels, output_size):
    """
    Convert labels to a format that matches the model's output.
    Assumes labels are in shape [batch_size, 1, 30, 30].
    """
    #flatten the spatial dimensions
    labels_flat = labels.view(labels.size(0), -1)  # [batch_size, 900]
    
    #optionally, aggregate labels if needed (e.g., mean)
    labels_agg = labels_flat.mean(dim=1, keepdim=True)  #reduce to [batch_size, 1]

    #expand to match output size if needed
    labels_expanded = labels_agg.expand(-1, output_size)  # [batch_size, output_size]
    
    return labels_expanded

#function to train the model
def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            optimizer.zero_grad()
            
            #forward pass
            outputs = model(inputs)
            
            #unpack outputs
            integration_output = outputs[-1]  #assuming the last output is the integration_output
            
            #print shapes for debugging
            print(f"Integration input size: {inputs.shape}")
            print(f"Integration output shape: {integration_output.shape}")
            print(f"Labels shape: {labels.shape}")
            
            #process labels to match the integration output size
            processed_labels = process_labels(labels, integration_output.size(1))
            
            #print processed label shape for debugging
            print(f"Processed labels shape: {processed_labels.shape}")
            
            #compute loss
            loss = criterion(integration_output, processed_labels)
            
            #backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        average_loss = running_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}")

#function to evaluate the model
def evaluate_model(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in dataloader:
            #forward pass
            outputs = model(inputs)
            
            #unpack outputs
            integration_output = outputs[-1]  #assuming the last output is the integration_output
            
            #print shapes for debugging
            print(f"Integration input size: {inputs.shape}")
            print(f"Integration output shape: {integration_output.shape}")
            print(f"Labels shape: {labels.shape}")
            
            #process labels to match the integration output size
            processed_labels = process_labels(labels, integration_output.size(1))
            
            #print processed label shape for debugging
            print(f"Processed labels shape: {processed_labels.shape}")
            
            #compute loss
            loss = criterion(integration_output, processed_labels)
            
            running_loss += loss.item()
    
    average_loss = running_loss / len(dataloader)
    print(f"Evaluation Loss: {average_loss:.4f}")

#example usage
if __name__ == "__main__":
    #load data
    training_loader = DataLoader(training_dataset, batch_size=8, shuffle=True)
    evaluation_loader = DataLoader(evaluation_dataset, batch_size=8, shuffle=False)
    
    #define criterion and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(yambi_model.parameters(), lr=0.001)
    
    #train Yambi model
    train_model(yambi_model, training_loader, criterion, optimizer, num_epochs=10)
    
    #evaluate Yambi model
    evaluate_model(yambi_model, evaluation_loader, criterion)

Reasoning output size: torch.Size([8, 128])
Compassionate output size: torch.Size([8, 128])
Bias Detection output size: torch.Size([8, 128])
Planning output size: torch.Size([8, 128])
Attention output size: torch.Size([8, 128])
Learning output size: torch.Size([8, 128])
Meta-Reasoning output size: torch.Size([8, 10])
Communication output size: torch.Size([8, 16])
Integration input size: torch.Size([8, 794])
Integration input size: torch.Size([8, 1, 30, 30])
Integration output shape: torch.Size([8, 900])
Labels shape: torch.Size([8, 1, 30, 30])
Processed labels shape: torch.Size([8, 900])
Reasoning output size: torch.Size([8, 128])
Compassionate output size: torch.Size([8, 128])
Bias Detection output size: torch.Size([8, 128])
Planning output size: torch.Size([8, 128])
Attention output size: torch.Size([8, 128])
Learning output size: torch.Size([8, 128])
Meta-Reasoning output size: torch.Size([8, 10])
Communication output size: torch.Size([8, 16])
Integration input size: torch.Size([8, 

In [32]:
#define the path to test data
test_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json'

#load test data
def load_json_data(file_path):
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        #print out some information for debugging purposes
        print(f"Successfully loaded data from {file_path}.")
        print(f"Number of keys in the dataset: {len(data.keys())}")
        return data
    except FileNotFoundError:
        print(f"Error: The file {file_path} does not exist.")
    except json.JSONDecodeError:
        print(f"Error: The file {file_path} is not a valid JSON file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

test_challenges = load_json_data(test_challenges_path)

Successfully loaded data from /kaggle/input/arc-prize-2024/arc-agi_test_challenges.json.
Number of keys in the dataset: 100


In [33]:
#define test dataset
class TestDataset(Dataset):
    def __init__(self, data, target_size=(30, 30), transform=None):
        self.data = data
        self.transform = transform
        self.target_size = target_size
        self.keys = list(data.keys())

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        key = self.keys[idx]
        test_cases = self.data[key]['test']
        #assuming we want to process the first test case
        grid = test_cases[0]['input']
        grid = np.array(grid).astype(np.float32)  #convert to numpy array
        grid = np.expand_dims(grid, axis=0)  #add channel dimension

        # Pad grid to the target size
        grid = self.pad_to_size(grid, self.target_size)

        grid = torch.tensor(grid)  #convert to PyTorch tensor
        if self.transform:
            grid = self.transform(grid)
        return grid, key

    def pad_to_size(self, grid, size):
        _, h, w = grid.shape
        target_h, target_w = size
        pad_h = max(0, target_h - h)
        pad_w = max(0, target_w - w)
        padding = (0, pad_w, 0, pad_h)
        #pad using torch tensor and convert back to numpy
        padded_grid = F.pad(torch.tensor(grid), padding, mode='constant', value=0).numpy()
        return padded_grid

In [34]:
#define grid dimensions
grid_height = 30
grid_width = 30

#create the dataset and dataloader for the test data
test_dataset = TestDataset(data=test_challenges, target_size=(grid_height, grid_width))
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

#iterate over the DataLoader
for grids, keys in test_loader:
    print(f"Batch of grids shape: {grids.shape}")
    print(f"Keys: {keys}")

Batch of grids shape: torch.Size([8, 1, 30, 30])
Keys: ('007bbfb7', '00d62c1b', '017c7c7b', '025d127b', '045e512c', '0520fde7', '05269061', '05f2a901')
Batch of grids shape: torch.Size([8, 1, 30, 30])
Keys: ('06df4c85', '08ed6ac7', '09629e4f', '0962bcdd', '0a938d79', '0b148d64', '0ca9ddb6', '0d3d703e')
Batch of grids shape: torch.Size([8, 1, 30, 30])
Keys: ('0dfd9992', '0e206a2e', '10fcaaa3', '11852cab', '1190e5a7', '137eaa0f', '150deff5', '178fcbfb')
Batch of grids shape: torch.Size([8, 1, 30, 30])
Keys: ('1a07d186', '1b2d62fb', '1b60fb0c', '1bfc4729', '1c786137', '1caeab9d', '1cf80156', '1e0a9b12')
Batch of grids shape: torch.Size([8, 1, 30, 30])
Keys: ('1e32b0e9', '1f0c79e5', '1f642eb9', '1f85a75f', '1f876c06', '1fad071e', '2013d3e2', '2204b7a8')
Batch of grids shape: torch.Size([8, 1, 30, 30])
Keys: ('22168020', '22233c11', '2281f1f4', '228f6490', '22eb0ac0', '234bbc79', '23581191', '239be575')
Batch of grids shape: torch.Size([8, 1, 30, 30])
Keys: ('23b5c85d', '253bf280', '25d487e

In [35]:
#generate predictions
def generate_predictions(model, dataloader, output_file='/kaggle/working/submission.json'):
    predictions = {}
    
    model.eval()  #set the model to evaluation mode
    
    with torch.no_grad():
        for batch in dataloader:
            grids, keys = batch  #extract grids and corresponding keys from the batch
            
            #ensure the tensor has the correct shape [batch_size, channels, height, width]
            if grids.dim() == 4:  #check if the tensor is already in the correct shape
                grids = grids  #no need to reshape
            elif grids.dim() == 3:  #check if the tensor is missing the channel dimension
                grids = grids.unsqueeze(1)  #add the channel dimension
            
            #pass through the model to get predictions
            reasoning_output, compassionate_output, integration_output = model(grids)
            
            #iterate through each grid in the batch
            for i, key in enumerate(keys):
                grid = integration_output[i].detach().cpu().numpy().reshape((30, 30)).astype(int)
                
                #assuming the output grid should be in the same format for both attempts
                predictions[key] = [
                    {"attempt_1": grid.tolist(), "attempt_2": grid.tolist()}
                ]
    
    #save predictions to JSON file
    with open(output_file, 'w') as f:
        json.dump(predictions, f, indent=4)
    
    print(f"Submission file saved successfully as '{output_file}'.")

generate_predictions(yambi_model, test_loader)

Reasoning output size: torch.Size([8, 128])
Compassionate output size: torch.Size([8, 128])
Bias Detection output size: torch.Size([8, 128])
Planning output size: torch.Size([8, 128])
Attention output size: torch.Size([8, 128])
Learning output size: torch.Size([8, 128])
Meta-Reasoning output size: torch.Size([8, 10])
Communication output size: torch.Size([8, 16])
Integration input size: torch.Size([8, 794])
Reasoning output size: torch.Size([8, 128])
Compassionate output size: torch.Size([8, 128])
Bias Detection output size: torch.Size([8, 128])
Planning output size: torch.Size([8, 128])
Attention output size: torch.Size([8, 128])
Learning output size: torch.Size([8, 128])
Meta-Reasoning output size: torch.Size([8, 10])
Communication output size: torch.Size([8, 16])
Integration input size: torch.Size([8, 794])
Reasoning output size: torch.Size([8, 128])
Compassionate output size: torch.Size([8, 128])
Bias Detection output size: torch.Size([8, 128])
Planning output size: torch.Size([8, 

In [36]:
#extracting keys from dataloader
def extract_keys_from_dataloader(dataloader):
    keys_set = set()
    
    for batch in dataloader:
        # Assuming each batch is a tuple (grids, keys)
        grids, keys = batch
        keys_set.update(keys)
    
    return list(keys_set)

test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
expected_keys = extract_keys_from_dataloader(test_loader)
print(expected_keys)

['178fcbfb', '228f6490', '2bcee788', '25d487eb', '23b5c85d', '4093f84a', '2013d3e2', '00d62c1b', '445eab21', '0b148d64', '0d3d703e', '1b60fb0c', '0a938d79', '28e73c20', '3bdb4ada', '40853293', '11852cab', '09629e4f', '1f642eb9', '2dd70a9a', '3eda0437', '2c608aff', '150deff5', '27a28665', '3e980e27', '22168020', '3428a4f5', '41e4d17e', '4290ef0e', '23581191', '31aa019c', '1cf80156', '0dfd9992', '3c9b0459', '4258a5f9', '3befdf3e', '0962bcdd', '3345333e', '1c786137', '1a07d186', '1f85a75f', '1bfc4729', '007bbfb7', '2bee17df', '29c11459', '3bd67248', '239be575', '1caeab9d', '0520fde7', '1e0a9b12', '3631a71a', '2dc579da', '22233c11', '321b1fc6', '3ac3eb23', '272f95fa', '045e512c', '08ed6ac7', '264363fd', '3aa6fb7a', '05269061', '017c7c7b', '234bbc79', '444801d8', '28bf18c6', '1f876c06', '363442ee', '1b2d62fb', '22eb0ac0', '29623171', '39e1d7f9', '3f7978a0', '36d67576', '1e32b0e9', '137eaa0f', '1fad071e', '3af2c5a8', '25d8a9c8', '0e206a2e', '10fcaaa3', '36fdfd69', '25ff71a9', '29ec7d0e', '02

In [37]:
#validation of the submission file
def check_submission_file(file_path, expected_keys):
    try:
        with open(file_path, 'r') as file:
            submissions = json.load(file)
    except Exception as e:
        print(f"Error loading submission file: {e}")
        return
    
    num_entries = len(submissions)
    print(f"Number of entries in submission file: {num_entries}")
    
    if num_entries != len(expected_keys):
        print(f"Warning: The number of entries in the submission file does not match the expected number of keys.")
        return

    for key in expected_keys:
        if key not in submissions:
            print(f"Missing key: {key}")
            continue
        
        entry = submissions[key]
        if not isinstance(entry, list) or len(entry) != 1:
            print(f"Incorrect format for key {key}: Entry should be a list with one dictionary.")
            continue
        
        attempt_dict = entry[0]
        if not isinstance(attempt_dict, dict) or 'attempt_1' not in attempt_dict or 'attempt_2' not in attempt_dict:
            print(f"Incorrect format for key {key}: Entry dictionary should contain 'attempt_1' and 'attempt_2'.")
            continue
        
        attempt_1 = attempt_dict['attempt_1']
        attempt_2 = attempt_dict['attempt_2']
        
        if not (isinstance(attempt_1, list) and all(isinstance(row, list) and len(row) == 30 for row in attempt_1) and len(attempt_1) == 30):
            print(f"Incorrect format for key {key}: 'attempt_1' grid should be a 30x30 list of lists.")
        
        if not (isinstance(attempt_2, list) and all(isinstance(row, list) and len(row) == 30 for row in attempt_2) and len(attempt_2) == 30):
            print(f"Incorrect format for key {key}: 'attempt_2' grid should be a 30x30 list of lists.")

    print("Format check completed.")

#check the submission file
check_submission_file('/kaggle/working/submission.json', expected_keys)

Number of entries in submission file: 100
Format check completed.
